In [1]:
#游戏本体beta1.0
#调用的库
import pygame
import random
import sys

# 初始化Pygame
pygame.init()

# 游戏窗口设置
window_size = (800, 800)
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("沙盒游戏")

# 定义颜色
white = (255, 255, 255)
blue = (0, 0, 255)
black = (0, 0, 0)
bright_green = (66, 207, 58)
purple = (121, 49, 192)
blood_red = (204, 33, 36)  # 血红
pink_food = (237, 196, 219)  # 粉红色

# 玩家方块
player_size = 10
player_x = window_size[0] // 2 - player_size // 2
player_y = window_size[1] // 2 - player_size // 2
player_speed = 0.6
#游戏时间
clock = pygame.time.get_ticks()
#游戏开始时间
start_time = pygame.time.get_ticks()
# 基础设定
tree_numbers = 50
food_numbers = 30

# 创建地图
map_image = pygame.image.load("D://Ai_Project_Tool//Jupyter_notebook//遗传算法demo//map_demo//map_1.png")  # 加载地图图片
map_size = map_image.get_size()  # 获取地图图片的大小

# 玩家分数
player_score = 100

# 分数文本渲染
font = pygame.font.Font("D:\\Ai_Project_Tool\\Jupyter_notebook\\遗传算法demo\\阿里妈妈数黑体.ttf", 36)

# 初始饥饿值和生命值
hunger = 100
health = 100

# 饥饿值和生命值的扣除速度
hunger_rate = 0.001
damage_rate = 0.1
#饥饿值的增加速度
hunger_up_rate = 0.2
# 创建分数显示区域
info_display_rect = pygame.Rect(0, 0, window_size[0], window_size[1])
map_display_rect = pygame.Rect(0, 0, window_size[0], window_size[1])
#创建食物的列表，每个食物包含位置与标识
foods = []
for _ in range(food_numbers):
    food_x= random.randint(5,map_display_rect.width - 10)
    food_y = random.randint(5,map_display_rect.height - 10)
    food_id = len(foods)
    foods.append((food_x,food_y,food_id))
# 创建树的列表，每棵树包含位置和标识
trees = []
for _ in range(tree_numbers):
    tree_x = random.randint(5, map_display_rect.width - 10)
    tree_y = random.randint(5, map_display_rect.height - 10)
    tree_id = len(trees)  # 使用索引作为标识
    trees.append((tree_x, tree_y, tree_id))



def death_choice(player_x, player_y, window_size):
    if (player_x < 0 or player_x + player_size > map_display_rect.width or
            player_y < 0 or player_y + player_size > map_display_rect.height):
        return True
    return False

def generate_tree(x, y):
    tree_color = bright_green
    tree_size = 10
    pygame.draw.rect(screen, tree_color, (x, y, tree_size, tree_size))

def generate_food(x, y):
    food_color = pink_food
    food_size = 8
    pygame.draw.rect(screen,food_color,(x,y,food_size,food_size))
    
    

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    #获取当前时间
    current_time = pygame.time.get_ticks()
    #计算游戏时间
    game_time_ms = current_time -start_time
    game_time_seconds = game_time_ms //1000
    game_time_minutes = game_time_seconds //60
    # 格式化时间文本
    time_text = f"Time: {game_time_minutes:02d}:{game_time_seconds % 60:02d}"
    time_surface = font.render(time_text, True, black)
    #清空屏幕
    screen.fill((0,0,0))
    
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_x -= player_speed
    if keys[pygame.K_RIGHT]:
        player_x += player_speed
    if keys[pygame.K_UP]:
        player_y -= player_speed
    if keys[pygame.K_DOWN]:
        player_y += player_speed

    # 在信息显示区域渲染信息
    info_surface = screen.subsurface(info_display_rect)
    info_surface.fill(white)

    # 在地图显示区域绘制地图图片
    map_surface = screen.subsurface(map_display_rect)
    map_surface.blit(map_image, (0, 0))  # 绘制地图图片

    # 在信息显示区域绘制玩家分数和提示信息
    score_text = font.render("分数: " + str(player_score), True, black)
    info_surface.blit(score_text, (10, 10))

    # 在信息显示区域绘制饥饿值和生命值
    hunger_text = font.render("饥饿值: " + str(round(hunger, 2)), True, black)
    health_text = font.render("生命值: " + str(round(health, 2)), True, black)
    info_surface.blit(hunger_text, (10, 50))
    info_surface.blit(health_text, (10, 90))

    # 更新饥饿值
    hunger -= hunger_rate
    if hunger <= 0:
        hunger = 0
        # 扣除生命值
        health -= damage_rate

    # 判断生命值是否为0，若为0则游戏结束
    if health <= 0:
        print("游戏结束")
        pygame.quit()
        sys.exit()

    # 检查是否越界，若越界则扣除分数并重置玩家位置
    if death_choice(player_x, player_y, map_display_rect.size):
        player_score -= 10
        player_x = map_display_rect.width // 2 - player_size // 2
        player_y = map_display_rect.height // 2 - player_size // 2

    # 绘制玩家角色
    pygame.draw.rect(map_surface, purple, (player_x, player_y, player_size, player_size))

    
    #绘制食物，并检测与食物的碰撞
    collected_foods = []
    for food in foods:
        food_x,food_y,food_id = food
        generate_food(food_x,food_y)
        
        #检测玩家与食物的碰撞
        if (player_x +player_size >= food_x and player_x <= food_x +10 and
           player_y + player_size >= food_y and player_y <= food_y +10):
            hunger += hunger_up_rate
            collected_foods.append(food_id)#记录被收集的食物的标识
    #从食物的列表中移除已被收集的食物
    foods =[food for food in foods if food[2] not in collected_foods]
    # 绘制树，并检测与树的碰撞
    collected_trees = []
    for tree in trees:
        tree_x, tree_y, tree_id = tree
        generate_tree(tree_x, tree_y)

        # 检查玩家与树的碰撞
        if (player_x + player_size >= tree_x and player_x <= tree_x + 10 and
            player_y + player_size >= tree_y and player_y <= tree_y + 10):
            player_score += 10  # 收集树增加分数
            collected_trees.append(tree_id)  # 记录被收集的树的标识

    # 从树的列表中移除已被收集的树
    trees = [tree for tree in trees if tree[2] not in collected_trees]

   
        # 游戏胜利条件
    if player_score >= 500:
        print("游戏胜利")
        pygame.quit()
        sys.exit()

    # 获取文本的宽度和高度
    text_width, text_height = time_surface.get_size()

    # 在窗口中央绘制时间文本
    screen.blit(time_surface, (window_size[0] // 30+250 - text_width // 30, window_size[1] // 40 - text_height //10))
    pygame.display.update(map_display_rect)



pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

D:\Ai_Project_Tool\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
